# Creating an AI newsletter assistant with Composio and Groq

This notebook demonstrates how to easily connect AI agents to external applications like Gmail, Slack, GitHub and many more using Composio. More specifically, we'll be setting up an assistant that will automatically look for email newsletters in our inbox, summarize and enrich their content with contextually relevant information, and then send these summaries to us in a well-formatted and easy to digest email.

### About Composio

Composio is a toolset designed to streamline and optimize the connection and interaction between AI Agents/LLMs and various APIs/services. Composio supports integrations with a variety of popular apps including Google Apps, Github, Slack, and also system tools like Code Interpreter, File Manager and Databases, in just a few lines of code. It also provides pre-defined actions(functions), allowing agents to interact with these integrations.

Let's get started!

### Setup

Install necessary packages -

In [ ]:
!pip install langchain langchain-core langchain-groq composio-langgraph langgraph

In [9]:
# Import necessary packages
from typing import Literal, Annotated, Sequence, TypedDict
import operator
import os
from datetime import datetime
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import ToolNode
from composio_langgraph import Action, ComposioToolSet

# Get you Groq API key here - https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = "your_groq_api_key"

We are going to be using Composio's Gmail and Tavily tools in this guide. To use these tools, we first need to create integrations with these apps. This can be done using the command line. You'll need a Tavily API key, which you can get [here](https://app.tavily.com/).

To create an integration with Gmail, run the command below -

In [ ]:
!composio add gmail

Similarly, to create an integration with Tavily -

In [ ]:
!composio add tavily

In [13]:
# Initialize the model
llm = ChatGroq(model="llama-3.1-70b-versatile")

### Initialize Toolset and get tools

Now we initialize Composio's toolset which provides all the necessary tools and actions we need. For this guide, we will get the Gmail and Tavily tools from the toolset. The Gmail tools will allow the agent to perform actions like fetching and sending emails. The tavily tool allows the agent to search the web and gather more information about the topics of the newsletter.

In [ ]:
# Initialize the ComposioToolSet
composio_toolset = ComposioToolSet()

# Get the tools from the ComposioToolSet
gmail_tools = composio_toolset.get_tools(
    actions=[Action.GMAIL_SEND_EMAIL, Action.GMAIL_FETCH_EMAILS],
)

tavily_tool = composio_toolset.get_tools(
    actions=[Action.TAVILY_TAVILY_SEARCH],
)

tools = [*gmail_tools, *tavily_tool]

# Define Tool Node
tool_node = ToolNode(tools)

### Define State and Create Agents

Now, we define the state of the graph. This will just be a list of messages along with a key to track the most recent sender.

In [20]:
# This defines the state object that is passed between each node in the graph.
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

The following helper functions will help create agents. These agents will then be nodes in the graph.

In [22]:
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_message,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    return prompt | llm.bind_tools(tools)

We now define the nodes in the graph.

In [24]:
def create_agent_node(agent, name):
    def agent_node(state):
        result = agent.invoke(state)
        if not isinstance(result, ToolMessage):
            result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
        return {"messages": [result], "sender": name}
    return agent_node

email_fetcher_agent = create_agent(
    llm,
    gmail_tools,
    system_message=f"You are an expert at retrieving and organizing email content, with a keen eye for identifying newsletters. Your goal is to Fetch recent newsletter emails from the inbox. Please look for emails with the words 'newsletter' or 'digest' only for last 7 days. Today's date is {datetime.now().strftime('%B %d, %Y')}. Don't add any other unncessary filters. Pass these emails to email_summarizer_agent."
)

email_fetcher_node = create_agent_node(email_fetcher_agent, "email_fetcher")

email_summarizer_agent = create_agent(
    llm,
    tavily_tool,
    system_message="You are an expert in analyzing and summarizing complex information, with a talent for distilling essential points from various sources. Summarize the content of the fetched newsletter emails, highlighting key information and trends. Create a concise yet comprehensive summary highlighting the key points from each newsletter. Use tavily_tool to search for relevant information and include it in the summary if not already included. This summary will be used by email_sender."
)

email_summarizer_node = create_agent_node(email_summarizer_agent, "email_summarizer")

email_sender_agent = create_agent(
    llm,
    gmail_tools,
    system_message=f"""
        "You are an expert in composing and sending emails with well-formatted, visually appealing content. You have a knack for creating engaging subject lines and structuring information for easy readability. Send the summarized newsletter content using the Gmail API to 0arunabh30@gmail.com with a professional and engaging format."
        "Use the following structure for the email:\n\n"
        "Subject: Your Weekly News Digest - {datetime.now().strftime('%B %d, %Y')}\n\n"
        "<h1>Weekly News Digest</h1>\n\n"
        "<p>Dear Reader,</p>\n\n"
        "<p>Here's your curated summary of this week's top news items and insights:</p>\n\n"
        "[Insert summarized content here]\n\n"
        "Each main section should be separated by a horizontal rule, like this:\n"
        "<hr>\n\n"
        "Structure the content logically, with clear sections for each summarized newsletter or topic area.\n"
        "Use appropriate HTML formatting such as <strong> for headlines, "
        "<ul> and <li> for bullet points, and <br> for line breaks to enhance readability.\n\n"
        "Include a brief introduction at the beginning to set the context and a conclusion at the end "
        "to summarize the key takeaways and trends observed across the newsletters.\n\n"
        "<footer>\n"
        "<p>For more details on these stories, click on the provided links or stay tuned to our next update. "
        "<p>Best regards,<br>Your Newsletter Summary Team</p>\n"
        "</footer>\n\n"
        "Important: Ensure all HTML tags are properly closed and nested correctly."
    """
)

email_sender_node = create_agent_node(email_sender_agent, "email_sender")

### Define Edge logic

We can define some edge logic that is needed to decide what to do based on the results of the agents.

In [26]:
def router(state) -> Literal["call_tool", "__end__", "continue"]:
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        return "__end__"
    return "continue"

### Define the Graph

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("email_fetcher", email_fetcher_node)
workflow.add_node("email_summarizer", email_summarizer_node)
workflow.add_node("email_sender", email_sender_node)
workflow.add_node("call_tool", tool_node)

workflow.add_edge(START, "email_fetcher")
workflow.add_edge("email_sender" ,END)

workflow.add_conditional_edges(
    "email_fetcher",
    router,
    {"continue": "email_summarizer", "call_tool": "call_tool"},
)
workflow.add_conditional_edges(
    "email_summarizer",
    router,
    {"continue": "email_sender", "call_tool": "call_tool"},
)

workflow.add_conditional_edges(
    "email_sender",
    router,
    {"continue": END, "call_tool": "call_tool"},
)

workflow.add_conditional_edges(
    "call_tool",
    lambda x: x["sender"],
    {"email_fetcher": "email_fetcher",
     "email_summarizer": "email_summarizer",
     "email_sender": "email_sender",
    },
)

app = workflow.compile()

# Invoke the graph!

events = app.invoke(
    {
        "messages": [
            HumanMessage(
                content=""
                "Please begin."
            )
        ],
    },
)

print(events)

### Conclusion

In this guide, we successfully used Groq to connect to the LLM and Composio allowed us to easily connect to Gmail and Tavily and use their tools to create an AI agent that can fetch newsletter emails, summarize them and then send the summary as a mail to the user. We used Langgraph to orchestrate a multi-agent system where each agent performs a specific task with the tools provided.

### Connect with us

Join the [Composio Discord](https://discord.com/invite/cNruWaAhQk) and [Groq Discord](https://discord.com/invite/groq) servers to discuss your ideas and get support from the community.

### Learn More

Composio: https://composio.dev/

Composio Docs: https://docs.composio.dev/introduction/intro/overview